# 🎯 HAI Deepfake - 데이터 준비 노트북 (Colab)

**이 노트북의 목적:**
1. ✅ Kaggle 데이터셋을 Google Drive로 다운로드
2. ✅ 비디오 → 이미지 프레임 추출
3. ✅ 소규모 데이터셋 생성 (빠른 테스트용)
4. ✅ 데이터 검증

**작업 순서:**
- 단계별로 셀을 실행하세요
- 첫 실행 시 전체 실행 (약 30분~1시간)
- 이후에는 필요한 섹션만 실행

---
## 📌 Step 0: 환경 확인

In [ ]:
import torch
import sys

print("="*70)
print("🔍 환경 정보")
print("="*70)
print(f"Python: {sys.version.split()[0]}")
print(f"PyTorch: {torch.__version__}")

if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA: {torch.version.cuda}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  GPU 없음 - '런타임 > 런타임 유형 변경 > GPU' 선택하세요!")
print("="*70)

---
## 📌 Step 1: Google Drive 마운트

In [ ]:
from google.colab import drive
import os
from pathlib import Path

# Google Drive 마운트
drive.mount('/content/drive')

# 프로젝트 루트 설정
DRIVE_ROOT = Path("/content/drive/MyDrive/HAI_Deepfake")
DRIVE_ROOT.mkdir(exist_ok=True)

# 하위 디렉토리 생성
(DRIVE_ROOT / "datasets").mkdir(exist_ok=True)
(DRIVE_ROOT / "train_data").mkdir(exist_ok=True)
(DRIVE_ROOT / "train_data_small").mkdir(exist_ok=True)
(DRIVE_ROOT / "models").mkdir(exist_ok=True)
(DRIVE_ROOT / "checkpoints").mkdir(exist_ok=True)

print("✅ Google Drive 마운트 완료!")
print(f"📁 프로젝트 루트: {DRIVE_ROOT}")
print(f"\n📂 디렉토리 구조:")
print(f"  {DRIVE_ROOT}/")
print(f"  ├── datasets/         # Kaggle 원본 데이터")
print(f"  ├── train_data/       # 전처리된 학습 데이터")
print(f"  ├── train_data_small/ # 소규모 테스트 데이터")
print(f"  ├── models/           # 최종 모델")
print(f"  └── checkpoints/      # 학습 체크포인트")

---
## 📌 Step 2: 프로젝트 코드 동기화 (Git)

In [ ]:
REPO_URL = "https://github.com/CBottle/HAI_Deepfake.git"
PROJECT_DIR = "/content/HAI_Deepfake"

if os.path.exists(PROJECT_DIR):
    print("🔄 기존 코드 업데이트 중...")
    %cd {PROJECT_DIR}
    !git fetch --all
    !git reset --hard origin/main
    print("✅ 코드 업데이트 완료!")
else:
    print("📥 프로젝트 클론 중...")
    !git clone {REPO_URL} {PROJECT_DIR}
    %cd {PROJECT_DIR}
    print("✅ 클론 완료!")

# 작업 디렉토리 확인
print(f"\n📍 현재 위치: {os.getcwd()}")
!ls -la

---
## 📌 Step 3: 필요한 라이브러리 설치

In [ ]:
print("📦 라이브러리 설치 중...\n")

# requirements.txt가 있으면 사용, 없으면 직접 설치
if os.path.exists("env/requirements.txt"):
    !pip install -q -r env/requirements.txt
else:
    !pip install -q kaggle opencv-python tqdm pillow pandas scikit-learn

print("\n✅ 설치 완료!")

---
## 📌 Step 4: Kaggle API 설정

**⚠️ 중요: 처음 실행 시 kaggle.json 업로드 필요**

### Kaggle API 토큰 받는 방법:
1. https://www.kaggle.com/settings 접속
2. "Create New API Token" 클릭
3. 다운로드된 `kaggle.json` 파일을 Google Drive에 업로드:
   - 위치: `/MyDrive/HAI_Deepfake/kaggle.json`

In [ ]:
import shutil

# kaggle.json 경로
kaggle_json_drive = DRIVE_ROOT / "kaggle.json"
kaggle_json_local = Path.home() / ".kaggle" / "kaggle.json"

if kaggle_json_drive.exists():
    print("✅ kaggle.json 파일 발견!")
    
    # ~/.kaggle 디렉토리 생성
    kaggle_json_local.parent.mkdir(exist_ok=True)
    
    # 복사
    shutil.copy(kaggle_json_drive, kaggle_json_local)
    
    # 권한 설정
    os.chmod(kaggle_json_local, 0o600)
    
    print("✅ Kaggle API 설정 완료!")
    
    # 테스트
    !kaggle datasets list --max-size 1000 | head -5
    
else:
    print("❌ kaggle.json 파일을 찾을 수 없습니다!")
    print(f"\n📝 다음 경로에 업로드하세요:")
    print(f"   {kaggle_json_drive}")
    print(f"\n💡 Kaggle API 토큰 받는 방법:")
    print(f"   1. https://www.kaggle.com/settings")
    print(f"   2. 'Create New API Token' 클릭")
    print(f"   3. 다운로드된 kaggle.json을 위 경로에 업로드")

---
## 📌 Step 5: 데이터셋 다운로드

### 추천 데이터셋:
1. **FaceForensics++** (~10GB) - 소규모 시작용
2. **CelebA** (~1.5GB) - Real 얼굴 추가용
3. **DFDC** (~470GB) - 대규모 (선택)

In [ ]:
# 데이터 다운로드 스크립트 실행
%run scripts/download_datasets.py

# downloader 객체 사용 가능

In [ ]:
# 옵션 1: FaceForensics++ 다운로드 (소규모 시작 - 추천)
print("📥 FaceForensics++ 다운로드 시작...")
print("⏱️  예상 시간: 10~30분 (네트워크 속도에 따라 다름)\n")

# downloader.download_dataset(
#     dataset_name="sorokin/faceforensics",
#     output_name="faceforensics"
# )

print("\n⚠️  주석을 해제하고 실행하세요!")

In [ ]:
# 옵션 2: CelebA 추가 다운로드 (Real 이미지용)
# downloader.download_dataset(
#     dataset_name="jessicali9530/celeba-dataset",
#     output_name="celeba"
# )

print("⚠️  Real 이미지가 더 필요하면 주석 해제하고 실행하세요!")

In [ ]:
# 현재 다운로드된 데이터셋 확인
info = downloader.get_dataset_info()

print("\n" + "="*70)
print("📊 다운로드된 데이터셋")
print("="*70)

if info:
    for name, details in info.items():
        print(f"✅ {name}:")
        print(f"   📁 경로: {details['path']}")
        print(f"   📄 파일: {details['file_count']:,d} 개")
        print(f"   💾 크기: {details['size_gb']:.2f} GB")
        print()
else:
    print("📭 아직 다운로드된 데이터셋이 없습니다.")
    print("위 셀에서 데이터셋을 다운로드하세요!")

print("="*70)

---
## 📌 Step 6: 비디오 → 이미지 변환

다운로드된 비디오에서 프레임을 추출하여 이미지로 저장합니다.

In [ ]:
# 프레임 추출 설정
INPUT_DATASET = str(DRIVE_ROOT / "datasets" / "faceforensics")  # 입력 데이터셋
OUTPUT_DIR = str(DRIVE_ROOT / "train_data")  # 출력 디렉토리
MAX_FRAMES = 30  # 비디오당 추출할 프레임 수
MAX_VIDEOS = None  # 테스트용: 숫자 지정 시 해당 개수만 처리 (예: 10)

print(f"📂 입력: {INPUT_DATASET}")
print(f"📁 출력: {OUTPUT_DIR}")
print(f"⚙️  설정: 비디오당 {MAX_FRAMES} 프레임")

if MAX_VIDEOS:
    print(f"⚠️  테스트 모드: {MAX_VIDEOS}개 비디오만 처리")

In [ ]:
# 프레임 추출 실행
!python scripts/extract_frames.py \
    --input "{INPUT_DATASET}" \
    --output "{OUTPUT_DIR}" \
    --max-frames {MAX_FRAMES} \
    --sample-method uniform \
    --quality 95

---
## 📌 Step 7: 소규모 데이터셋 생성 (빠른 테스트용)

전체 데이터로 학습하기 전에 작은 데이터셋으로 먼저 테스트합니다.

In [ ]:
# 소규모 데이터셋 설정
SMALL_INPUT = str(DRIVE_ROOT / "train_data")
SMALL_OUTPUT = str(DRIVE_ROOT / "train_data_small")
NUM_SAMPLES = 1000  # 클래스당 샘플 수 (Real 1000 + Fake 1000 = 총 2000)

print(f"📂 원본: {SMALL_INPUT}")
print(f"📁 출력: {SMALL_OUTPUT}")
print(f"📊 샘플: 클래스당 {NUM_SAMPLES:,d} 개")

In [ ]:
# 소규모 데이터셋 생성
!python scripts/create_small_dataset.py \
    --input "{SMALL_INPUT}" \
    --output "{SMALL_OUTPUT}" \
    --num-samples {NUM_SAMPLES} \
    --seed 42

---
## 📌 Step 8: 데이터 검증

생성된 데이터셋을 확인하고 샘플 이미지를 봅니다.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import random

def show_samples(data_dir, num_samples=6):
    """데이터셋 샘플 시각화"""
    data_path = Path(data_dir)
    
    fig, axes = plt.subplots(2, num_samples, figsize=(15, 5))
    fig.suptitle(f"Dataset Samples: {data_dir}", fontsize=16)
    
    for idx, label in enumerate(["real", "fake"]):
        label_dir = data_path / label
        images = list(label_dir.glob("*.jpg"))
        
        if len(images) == 0:
            print(f"⚠️  {label} 이미지가 없습니다.")
            continue
        
        samples = random.sample(images, min(num_samples, len(images)))
        
        for i, img_path in enumerate(samples):
            img = Image.open(img_path)
            axes[idx, i].imshow(img)
            axes[idx, i].axis('off')
            axes[idx, i].set_title(f"{label.upper()}", fontsize=10)
    
    plt.tight_layout()
    plt.show()

# 소규모 데이터셋 샘플 보기
show_samples(SMALL_OUTPUT, num_samples=6)

In [ ]:
# 데이터셋 통계
def print_dataset_stats(data_dir):
    """데이터셋 통계 출력"""
    data_path = Path(data_dir)
    
    print("\n" + "="*70)
    print(f"📊 데이터셋 통계: {data_dir}")
    print("="*70)
    
    total = 0
    for label in ["real", "fake"]:
        label_dir = data_path / label
        if label_dir.exists():
            images = list(label_dir.glob("*.jpg"))
            images += list(label_dir.glob("*.png"))
            count = len(images)
            total += count
            print(f"  {label.upper():5s}: {count:8,d} 이미지")
    
    print(f"  {'TOTAL':5s}: {total:8,d} 이미지")
    print("="*70)

# 전체 데이터셋
print_dataset_stats(SMALL_INPUT)

# 소규모 데이터셋
print_dataset_stats(SMALL_OUTPUT)

---
## ✅ 완료!

### 다음 단계:
1. **소규모로 학습 시작**: `train_colab.ipynb` 실행
2. **데이터 경로 설정**: 
   - 소규모 테스트: `/content/drive/MyDrive/HAI_Deepfake/train_data_small`
   - 전체 학습: `/content/drive/MyDrive/HAI_Deepfake/train_data`

### 저장된 위치:
- 📁 Google Drive: `/MyDrive/HAI_Deepfake/`
- 🎬 원본 비디오: `datasets/`
- 🖼️ 전체 이미지: `train_data/`
- 🧪 테스트 데이터: `train_data_small/`

### 팁:
- Colab 세션이 끊겨도 Google Drive 데이터는 안전!
- 다음 번에는 Step 1부터 바로 시작 가능
- 더 많은 데이터가 필요하면 Step 5로 돌아가서 추가 다운로드